In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [3]:
df_2018 = getDataframe(2018)

In [4]:
df_2018 = addTokenizedColumnofTitle(df_2018,None,False)

In [5]:
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)

In [6]:
words_2018.head()

,top,hacker,news,book,in,2017,beijing,enjoy,best,winter,...,400gb,shareseer,öyster,face-detecting,rainer,rilke,computing.foundation,integer32,nichols,goulding
story,1277,1048,1424,1283,33965,2091,81,49,1716,147,...,1,1,1,1,1,1,1,1,1,2
ask_hn,37,86,111,249,1420,22,1,10,648,2,...,0,0,0,0,0,0,0,0,0,0
show_hn,68,100,184,77,1149,6,0,1,57,2,...,0,0,0,0,0,0,0,0,0,0
poll,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def getWordListBasedOnCount(words_df,maxCount = 1):
    wordList = []
    for word in list(words_df.columns.values):
        count = np.sum(words_df[word])
        if count <= maxCount:
            wordList.append(word)
    return wordList
stopWordList = getWordListBasedOnCount(words_2018,maxCount = 1)
print(stopWordList[0],len(stopWordList))

registrars 51873


In [11]:
df_2018 = addTokenizedColumnofTitle(df_2018,stopWordList,False)
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)
words_2018.head()

,top,hacker,news,book,in,2017,beijing,enjoy,best,winter,...,months-long,joled,21.6-inch,kotlin/coroutines,osfp,nervos,vimes,streamy-db,fake-porn,goulding
story,1277,1048,1424,1283,33965,2091,81,49,1716,147,...,2,2,2,2,2,2,2,1,3,2
ask_hn,37,86,111,249,1420,22,1,10,648,2,...,0,0,0,0,0,0,0,0,0,0
show_hn,68,100,184,77,1149,6,0,1,57,2,...,0,0,0,0,0,0,0,1,0,0
poll,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_2018.head()

,Title,Post Type,Number of Comments,Points,Author,tokenized_title
0,Top Hacker News Books in 2017,story,0.0,1,0x54MUR41,"[top, hacker, news, book, in, 2017]"
1,Beijing enjoys best winter air quality in five...,story,0.0,2,gpetukhov,"[beijing, enjoy, best, winter, air, quality, i..."
2,Ask HN: Any domain name registrars that don't ...,ask_hn,2.0,1,glockenspielen,"[ask, hn, any, domain, name, that, do, n't, re..."
3,Controversial Therapy Has Led to Death Threats...,story,0.0,2,cpncrunch,"[controversial, therapy, have, lead, to, death..."
4,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,0.0,2,geraldbauer,"[ruby, 3x3, ruby, 3, will, be, 3, time, faster..."


In [13]:
priorProbabilities = getPriorProbabilities(df_2018)
words_2018 = obtainDataframeWithClassProbabilities(words_2018, AllClasses, delta, appendClassPrefix)
writeModel(words_2018,'wordlength-model.txt',AllClasses,appendClassPrefix)
words_2018 = renameModelRows(words_2018, AllClasses, appendClassPrefix)
model = [words_2018, priorProbabilities]

In [14]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData,stopWordList,False)
testData = testData.reset_index()
testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'wordlength-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)

0.9710017126407463
predicted  ask_hn  poll  show_hn   story
Post Type                               
ask_hn       5421     0       11      22
poll            2     0        0       4
show_hn        96     0     4740      67
story        1616     0     2160  123075
predicted  ask_hn  poll  show_hn   story  precision    recall  F1_score
Post Type                                                              
ask_hn       5421     0       11      22   0.759776  0.993949  0.861228
poll            2     0        0       4   0.000000  0.000000  0.000000
show_hn        96     0     4740      67   0.685863  0.966755  0.802438
story        1616     0     2160  123075   0.999245  0.970233  0.984525


/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
